In [1]:
import requests
import json
from pprint import pprint
import pandas as pd
import os 
import sqlite3
from credentials import key

In [2]:
# # api call for first half of results

# # set up the request parameters
# params_1 = {
#     'api_key': key,
#     'search_term': 'laundry detergent',
#     'category_id': 'uqe9q',
#     'type': 'search',
#     'page': '1',
#     'max_page': '5'
# }

# # make the http GET request to RedCircle API
# api_result_1 = requests.get('https://api.redcircleapi.com/request', params_1)

# # print the JSON response from RedCircle API
# pprint(json.dumps(api_result_1.json()))
# json_data_1 = api_result_1.json()

In [3]:
# # api call for second half of results


# # set up the request parameters
# params_2 = {
#     'api_key': key,
#     'search_term': 'laundry detergent',
#     'category_id': 'uqe9q',
#     'type': 'search',
#     'max_page': '8',
#     'page': '6'
# }


# # make the http GET request to RedCircle API
# api_result_2 = requests.get('https://api.redcircleapi.com/request', params_2)

# # print the JSON response from RedCircle API
# pprint(json.dumps(api_result_2.json()))
# json_data_2 = api_result_2.json()

In [4]:
# # save the 1st search results json in a .json file
# out_file = open("resources/search-results-1.json", "w")
  
# json.dump(json_data_1, out_file, indent = 4)
  
# out_file.close()

# # save the 2nd search results json in a .json file
# out_file = open("resources/search-results-2.json", "w")
  
# json.dump(json_data_2, out_file, indent = 4)
  
# out_file.close()

In [5]:
# read the 2 json search results as dicts

# read the first json files
f = open('resources/search-results-1.json')
# returns JSON object as a dictionary
search_data_1 = json.load(f)
# Closing file
f.close()

# read the first json files
f = open('resources/search-results-2.json')
# returns JSON object as a dictionary
search_data_2 = json.load(f)
# Closing file
f.close()

In [6]:
# create dataframe from 2 search results dicts
search_results_df_1 = pd.DataFrame(search_data_1['search_results'])
search_results_df_2 = pd.DataFrame(search_data_2['search_results'])
search_results_df = search_results_df_1.append(search_results_df_2, ignore_index=True)

search_results_df.head()

,position,product,offers,fulfillment,page,position_overall,seller
0,1,{'title': 'Gain + Aroma Boost Original Scent H...,"{'primary': {'price': 15.99, 'currency': 'USD'...",{'type': '1p'},1,1,NaN
1,2,{'title': 'Gain flings! Laundry Detergent Pacs...,"{'primary': {'price': 27.49, 'currency': 'USD'...",{'type': '1p'},1,2,NaN
2,3,{'title': 'Free Clear HE Liquid Laundry Deterg...,"{'primary': {'price': 7.69, 'currency': 'USD',...",{'type': '1p'},1,3,NaN
3,4,{'title': 'Arm &#38; Hammer Plus OxiClean Fre...,"{'primary': {'price': 14.69, 'currency': 'USD'...",{'type': '1p'},1,4,NaN
4,5,{'title': 'All Free Clear Liquid Concentrated ...,"{'primary': {'price': 13.29, 'currency': 'USD'...",{'type': '1p'},1,5,NaN


In [7]:
# Each column is a dictionary. Get the dictionary of product data from the 'product' column.
product_json = search_results_df.loc[8]['product']

# Each column is a dictionary. Get the dictionary of price data from the 'offers'column.
offers_json = search_results_df.loc[8]['offers']
# The price information is in the 'primary' key of the offers_json
price_json = offers_json['primary']
pprint(product_json)

{'brand': 'Everspring',
 'brand_link': 'https://www.target.com/b/everspring/-/N-gl7n0',
 'class_id': 2,
 'department_id': 3,
 'dpci': '003-02-6943',
 'feature_bullets': ['64 loads',
                     '2x concentrated',
                     'Removes the toughest stains',
                     '7 powerful stain-fighting enzymes',
                     '100% natural fragrance',
                     'Non-toxic to humans when used as directed',
                     'Not tested on animals',
                     '96% Biobased Product'],
 'images': ['https://target.scene7.com/is/image/Target/GUEST_66ac2473-06ad-44fe-8522-e8687e76c9ad',
            'https://target.scene7.com/is/image/Target/GUEST_921684f9-8b91-40d2-b0e8-123db90ed29d',
            'https://target.scene7.com/is/image/Target/GUEST_2c2b4b3d-8b9f-4972-9a33-73f4d60b0f37',
            'https://target.scene7.com/is/image/Target/GUEST_1123573a-34b3-427c-89b9-c6743e5d47c1'],
 'link': 'https://www.target.com/p/lavender-38-bergamot-liquid

In [8]:
# clean up the product dictionary for only what is needed
product_json = {
    'title': product_json['title'],
    'link': product_json['link'],
    'department_id': product_json['department_id'],
    'brand': product_json['brand'],
    'rating':product_json['rating'],
    'tcin': product_json['tcin'],
    'position': search_results_df.loc[8]['position']
}
product_json

{'title': 'Lavender &#38; Bergamot Liquid Laundry Detergent - 100 fl oz - Everspring&#8482;',
 'link': 'https://www.target.com/p/lavender-38-bergamot-liquid-laundry-detergent-100-fl-oz-everspring-8482/-/A-75663151',
 'department_id': 3,
 'brand': 'Everspring',
 'rating': 4.4,
 'tcin': '75663151',
 'position': 9}

In [9]:
# For each row of the search_results_df, make a dict for product_json and price_json and add it to the full_product_df.
price_x_product_df = pd.DataFrame(

                        {'title': pd.Series(dtype='str'),
                        'link': pd.Series(dtype='str'),
                        'department_id': pd.Series(dtype='int'),
                        'brand': pd.Series(dtype='str'),
                        'rating': pd.Series(dtype='float'),
                        'tcin': pd.Series(dtype='str'),
                        'price': pd.Series(dtype='float'),
                        'currency': pd.Series(dtype='str'),
                        'symbol': pd.Series(dtype='str'),
                        'sponsored': pd.Series(dtype='str')
                        }
                    )

for i in range(len(search_results_df)):

    product_json = search_results_df.loc[i]['product']
    price_json = search_results_df.loc[i]['offers']['primary']
    position = search_results_df.loc[i]['position']
    if 'rating' in product_json:
        r = product_json['rating']
    else:
        r = 0
    if 'sponsored' in product_json:
        s = product_json['sponsored']
    else:
        s = 'False'

    full_product_info = {
        'title': product_json['title'],
        'link': product_json['link'],
        'department_id': product_json['department_id'],
        'brand': product_json['brand'],
        'rating': r,
        'tcin': product_json['tcin'],
        'price': price_json['price'], 
        'currency': price_json['currency'], 
        'symbol': price_json['symbol'],
        'sponsored': s
    }

    price_x_product_df = price_x_product_df.append(full_product_info, ignore_index=True)

price_x_product_df


,title,link,department_id,brand,rating,tcin,price,currency,symbol,sponsored
0,Gain + Aroma Boost Original Scent HE Compatibl...,https://www.target.com/p/gain-aroma-boost-orig...,3,Gain,0.0,78016139,15.99,USD,$,True
1,Gain flings! Laundry Detergent Pacs - Original...,https://www.target.com/p/gain-flings-laundry-d...,3,Gain,0.0,84114522,27.49,USD,$,True
2,Free Clear HE Liquid Laundry Detergent - 100 f...,https://www.target.com/p/free-clear-he-liquid-...,3,up & up,0.0,13186521,7.69,USD,$,False
3,Arm &#38; Hammer Plus OxiClean Fresh Scent Li...,https://www.target.com/p/arm-38-hammer-plus-ox...,3,Arm & Hammer,0.0,47054194,14.69,USD,$,False
4,All Free Clear Liquid Concentrated Laundry Con...,https://www.target.com/p/all-free-clear-liquid...,3,All,0.0,48638959,13.29,USD,$,False
...,...,...,...,...,...,...,...,...,...,...
194,Grab Green Garbage Disposal Freshener & Cleane...,https://www.target.com/p/grab-green-garbage-di...,3,Grab Green,0.0,88332246,5.95,USD,$,False
195,Grab Green Newborn Baby Laundry Detergent Pods...,https://www.target.com/p/grab-green-newborn-ba...,3,Grab Green,0.0,88267094,11.89,USD,$,False
196,"Grab Green 3 in 1 Laundry Detergent Pods, Pouc...",https://www.target.com/p/grab-green-3-in-1-lau...,3,Grab Green,0.0,88332226,12.99,USD,$,False
197,"Grab Green 3 in 1 Laundry Detergent Pods, Jumb...",https://www.target.com/p/grab-green-3-in-1-lau...,3,Grab Green,0.0,88332221,42.75,USD,$,False


In [20]:
# find duplicated titles
s = pd.Series(price_x_product_df.duplicated(subset=['title'], keep=False))
price_x_product_df2 = price_x_product_df
price_x_product_df2['duplicate_title'] = s
# print the duplicate title rows
price_x_product_df2 = price_x_product_df2.loc[price_x_product_df['duplicate_title'] == True]
price_x_product_df2

,title,link,department_id,brand,rating,tcin,price,currency,symbol,sponsored,duplicate_title
0,Gain + Aroma Boost Original Scent HE Compatibl...,https://www.target.com/p/gain-aroma-boost-orig...,3,Gain,0.0,78016139,15.99,USD,$,True,True
1,Gain flings! Laundry Detergent Pacs - Original...,https://www.target.com/p/gain-flings-laundry-d...,3,Gain,0.0,84114522,27.49,USD,$,True,True
5,Gain + Aroma Boost Original Scent HE Compatibl...,https://www.target.com/p/gain-aroma-boost-orig...,3,Gain,0.0,78016139,15.99,USD,$,False,True
6,Gain flings! Liquid Laundry Detergent Pacs - M...,https://www.target.com/p/gain-flings-liquid-la...,3,Gain,0.0,84114501,27.49,USD,$,True,True
7,Gain Flings Super Fresh 3-in-1 with Febreze an...,https://www.target.com/p/gain-flings-super-fre...,3,Gain,0.0,84114502,21.49,USD,$,True,True
28,Gain flings! Laundry Detergent Pacs - Original...,https://www.target.com/p/gain-flings-laundry-d...,3,Gain,0.0,84114522,27.49,USD,$,True,True
29,Gain flings! Liquid Laundry Detergent Pacs - M...,https://www.target.com/p/gain-flings-liquid-la...,3,Gain,0.0,84114501,27.49,USD,$,True,True
34,Gain Flings Super Fresh 3-in-1 with Febreze an...,https://www.target.com/p/gain-flings-super-fre...,3,Gain,0.0,84114502,21.49,USD,$,True,True
55,Gain flings! Liquid Laundry Detergent Pacs - M...,https://www.target.com/p/gain-flings-liquid-la...,3,Gain,0.0,84114501,27.49,USD,$,False,True
64,Gain flings! Laundry Detergent Pacs - Original...,https://www.target.com/p/gain-flings-laundry-d...,3,Gain,0.0,84114522,27.49,USD,$,False,True


In [19]:
price_x_product_df2.groupby(by=['title', 'sponsored']).count()

link  \
title                                              sponsored         
Gain + Aroma Boost Original Scent HE Compatible... True          1   
                                                   False         1   
Gain Flings Super Fresh 3-in-1 with Febreze and... True          2   
                                                   False         1   
Gain flings! Laundry Detergent Pacs - Original ... True          2   
                                                   False         1   
Gain flings! Liquid Laundry Detergent Pacs - Mo... True          2   
                                                   False         1   

                                                              department_id  \
title                                              sponsored                  
Gain + Aroma Boost Original Scent HE Compatible... True                   1   
                                                   False                  1   
Gain Flings Super Fresh 3-in-1 with Febreze and... True                   2   
                                                   False                  1   
Gain flings! Laundry Detergent Pacs - Original ... True                   2   
                                                   False                  1   
Gain flings! Liquid Laundry Detergent Pacs - Mo... True                   2   
                                                   False                  1   

                                                              brand  rating  \
title                                              sponsored                  
Gain + Aroma Boost Original Scent HE Compatible... True           1       1   
                                                   False          1       1   
Gain Flings Super Fresh 3-in-1 with Febreze and... True           2       2   
                                                   False          1       1   
Gain flings! Laundry Detergent Pacs - Original ... True           2       2   
                                                   False          1       1   
Gain flings! Liquid Laundry Detergent Pacs - Mo... True           2       2   
                                                   False          1       1   

                                                              tcin  price  \
title                                              sponsored                
Gain + Aroma Boost Original Scent HE Compatible... True          1      1   
                                                   False         1      1   
Gain Flings Super Fresh 3-in-1 with Febreze and... True          2      2   
                                                   False         1      1   
Gain flings! Laundry Detergent Pacs - Original ... True          2      2   
                                                   False         1      1   
Gain flings! Liquid Laundry Detergent Pacs - Mo... True          2      2   
                                                   False         1      1   

                                                              currency  \
title                                              sponsored             
Gain + Aroma Boost Original Scent HE Compatible... True              1   
                                                   False             1   
Gain Flings Super Fresh 3-in-1 with Febreze and... True              2   
                                                   False             1   
Gain flings! Laundry Detergent Pacs - Original ... True              2   
                                                   False             1   
Gain flings! Liquid Laundry Detergent Pacs - Mo... True              2   
                                                   False             1   

                                                              symbol  \
title                                              sponsored           
Gain + Aroma Boost Original Scent HE Compatible... True            1   
                                                   

In [21]:
# all the sponsored rows are duplicates, so drop them
price_x_product_df = price_x_product_df.loc[price_x_product_df['sponsored'] != True]
price_x_product_df

,title,link,department_id,brand,rating,tcin,price,currency,symbol,sponsored,duplicate_title
2,Free Clear HE Liquid Laundry Detergent - 100 f...,https://www.target.com/p/free-clear-he-liquid-...,3,up & up,0.0,13186521,7.69,USD,$,False,False
3,Arm &#38; Hammer Plus OxiClean Fresh Scent Li...,https://www.target.com/p/arm-38-hammer-plus-ox...,3,Arm & Hammer,0.0,47054194,14.69,USD,$,False,False
4,All Free Clear Liquid Concentrated Laundry Con...,https://www.target.com/p/all-free-clear-liquid...,3,All,0.0,48638959,13.29,USD,$,False,False
5,Gain + Aroma Boost Original Scent HE Compatibl...,https://www.target.com/p/gain-aroma-boost-orig...,3,Gain,0.0,78016139,15.99,USD,$,False,True
8,Lavender &#38; Bergamot Liquid Laundry Deterge...,https://www.target.com/p/lavender-38-bergamot-...,3,Everspring,4.4,75663151,11.99,USD,$,False,False
...,...,...,...,...,...,...,...,...,...,...,...
194,Grab Green Garbage Disposal Freshener & Cleane...,https://www.target.com/p/grab-green-garbage-di...,3,Grab Green,0.0,88332246,5.95,USD,$,False,False
195,Grab Green Newborn Baby Laundry Detergent Pods...,https://www.target.com/p/grab-green-newborn-ba...,3,Grab Green,0.0,88267094,11.89,USD,$,False,False
196,"Grab Green 3 in 1 Laundry Detergent Pods, Pouc...",https://www.target.com/p/grab-green-3-in-1-lau...,3,Grab Green,0.0,88332226,12.99,USD,$,False,False
197,"Grab Green 3 in 1 Laundry Detergent Pods, Jumb...",https://www.target.com/p/grab-green-3-in-1-lau...,3,Grab Green,0.0,88332221,42.75,USD,$,False,False


In [22]:
# save the search results in csv
os.makedirs('resources', exist_ok=True)  
price_x_product_df.to_csv('resources/price-x-product-data.csv')  

In [24]:
# create sqlite db
# conn = sqlite3.connect(':memory:')

conn = sqlite3.connect('laundry_detergent.db')

c = conn.cursor()

conn.execute("DROP TABLE laundry_detergents")

# create able
c.execute("""CREATE TABLE laundry_detergents(
        title TEXT,
        link TEXT,
        department_id INTEGER,
        brand TEXT,
        rating FLOAT,
        tcin TEXT,
        price REAL,
        currency TEXT,
        symbol TEXT
        position  INTEGER
    )""")

# add the price-x-product-data.csv data to table
data = pd.read_csv('resources/price-x-product-data.csv')
data.to_sql('laundry_detergents', conn, if_exists='replace', index=False)
# _________________________
# queries
# _________________________

c.execute("""
    SELECT t.c, t.title FROM
        (SELECT count(title) as c, title
        FROM laundry_detergents
        GROUP BY title) as t
    WHERE t.c > 1
    ORDER BY t.title
    """)

# c.execute("""
#     SELECT * FROM laundry_detergents
# """)

# _________________________
# end queries
# _________________________

c.execute
rows = c.fetchall()

conn.commit()

conn.close()

# check if any rows are not laundry detergent by seeing if the word in product title.
# for row in rows:
#     if 'Detergent' not in row[0]:
#         print(row[0])

# count of rows in table
for row in rows:
    print(row)

In [ ]:
# make serach api call for all the pages in the search results
# use the tcin to make api call for each product in the search results
